# RAG Q&A chatbot using document retrieval and generative AI for intelligent response generation (can use any light model from hugging face or a license llm(opneai, claude, grok, gemini) if free credits available

In [ ]:
import pandas as pd
df = pd.read_csv("Training Dataset.csv")
df = df.dropna()
documents = df.astype(str).apply(lambda row: ' | '.join(row), axis=1).tolist()
print("Sample document:\n", documents[0])

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(documents, convert_to_numpy=True)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

In [ ]:
def retrieve_top_docs(query, k=3):
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)
    return [documents[i] for i in indices[0]]

In [ ]:
import openai
openai.api_key = "YOUR_OPENAI_API_KEY" 
def generate_response(query, context_docs):
    context = "\n".join(context_docs)
    prompt = f"""Answer the question based only on the context below.
Context:
{context}
Question: {query}
Answer:"""
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    return response['choices'][0]['message']['content']

In [ ]:
while True:
    query = input("You: ")
    if query.lower() in ['exit', 'quit']:
        break
    top_docs = retrieve_top_docs(query)
    answer = generate_response(query, top_docs)
    print("\nBot:", answer)